In [2]:
from get_labelstudio_data import get_results_of_project, get_tasks
from helper import load_dotenv
from azure.storage.blob import BlobClient, ContainerClient
from datetime import datetime
from PIL import Image
import io
import requests
from tqdm.auto import tqdm

In [26]:
from file_interaction import get_batch_files

In [29]:
missing_masks = pd.concat(
    [pickle.loads( download_blob(bf).getbuffer() ) for bf in get_batch_files( batch_type='masks' )],
    ignore_index=True
)

In [30]:
missing_masks

,filename,job,screen_ruling,import_timestamp,origin,file_available
0,301297.F_Kalender_2013.p1,24-02-12_randomPages,NaN,2024-02-12 18:37:47.629565,None,False
1,147874.tg51_01-80_9783328300335_1.0_INH_Girod_...,24-02-12_randomPages,NaN,2024-02-12 18:37:47.629565,None,False
2,150104.tg39_huelsen_bildpreisliste_2024_image_...,24-02-12_randomPages,NaN,2024-02-12 18:37:47.629565,None,False
3,616350.tg39_pos1000_5347a_Flyer_RS_RZ.p1,24-02-12_randomPages,NaN,2024-02-12 18:37:47.629565,None,False
4,511155.tg42_441_tb_soest_2024_013_198_alpha.p124,24-02-12_randomPages,NaN,2024-02-12 18:37:47.629565,None,False
...,...,...,...,...,...,...
2049,510598.SportBild_XX_8_069_1.p1,24-03-05-01_randomTrainPages,NaN,2024-03-05 21:50:09.330758,None,False
2050,617141.tg40_2-28_Beilage_Weihnachten_2023_165x...,24-03-05-01_randomTrainPages,NaN,2024-03-05 21:50:09.330758,None,False
2051,510108.g41_ZM_2024_11_0002_V1.p1,24-03-05-01_randomTrainPages,NaN,2024-03-05 21:50:09.330758,None,False
2052,149645.IO43_gmg-konv43_9783064203778 Inhalt_S0...,24-03-05-01_randomTrainPages,NaN,2024-03-05 21:50:09.330758,None,False


In [3]:
dotenv = load_dotenv()

In [ ]:
def create_annotation( target_task, results ):
    with Image.open( download_blob(target_task['data_path']) ) as img:
        img_size = img.size
     
    headers = {'Authorization': f'Token { dotenv["LABEL_STUDIO_TOKEN"] }'}
    json_data = {
        "completed_by": 1,
        "result": [
            {
                "original_width": img_size[0],
                "original_height": img_size[1],
                "image_rotation": 0,
                "value": {
                    "x": r['value']['x'] / img_size[0] * 100,
                    "y": r['value']['y'] / img_size[1] * 100,
                    "width": r['value']['width'] / img_size[0] * 100,
                    "height": r['value']['height'] / img_size[1] * 100,
                    "rotation": 0,
                    "rectanglelabels": r['rectanglelabels']
                },
                "from_name": "label",
                "to_name": "image",
                "type": "rectanglelabels",
                "origin": "manual"
            }
            for r in results
        ],
        "was_cancelled": False,
        "ground_truth": False,
        "draft_created_at": datetime.now().isoformat(),
        "task": target_task["id"],
        "project": target_task["project_id"],
        "updated_by": 1,
        "parent_prediction": None,
        "parent_annotation": None
    }

    requests.post(
        f'http://localhost:8080/api/tasks/{ target_task["id"] }/annotations/',
        headers=headers,
        json=json_data
    )

In [ ]:
def upload_blob( filepath, prefix_path ):
    blob = BlobClient.from_connection_string(conn_str=dotenv['AZURE_CONNECTION_STRING'], container_name=dotenv['AZURE_CONTAINER_NAME'], blob_name=f"{ prefix_path }{ filepath.name }")

    with filepath.open("rb") as data:
         blob.upload_blob(
             data,
             connection_timeout=600
         )

In [ ]:
def download_blob( blob_name ):
    blob = BlobClient.from_connection_string(conn_str=dotenv['AZURE_CONNECTION_STRING'], container_name=dotenv['AZURE_CONTAINER_NAME'], blob_name=blob_name)
    stream = io.BytesIO()

    blob.download_blob().readinto(stream)
    return stream

In [ ]:
#Image.open(download_blob(relevant_blobs[505]))

In [ ]:
relevant_blobs = get_blobs(filter='labelstudio_moire_data/')
results = get_results_of_project(1)
tasks = get_tasks(1)
for t in tasks:
    t['img_name'] = t['data_path'].split('/')[-1]

results_by_img = {}

for r in results:
    if r['img_name'] not in results_by_img:
        results_by_img[r['img_name']] = {
            'local' : [],
            'azure' : []
        }

    results_by_img[r['img_name']][r['storage_type']].append(r)

for key in results_by_img:
    relevant_results = []

    if len(results_by_img[key]['azure']) == 0:
        relevant_results += results_by_img[key]['local']
    elif len(results_by_img[key]['local']) > len(results_by_img[key]['azure']):
        relevant_results += results_by_img[key]['local']

    results_by_img[key]['relevant_results'] = relevant_results

In [ ]:
from file_interaction import get_related_filepath, get_blobs

In [ ]:
from helper import load_dotenv

In [ ]:
0 / 0

In [ ]:
all_files = get_blobs( filter="data/")

In [ ]:
len(get_blobs( filter="data/148903/vps2400dpi150lpi/"))

In [ ]:
len(get_blobs( filter="data/148903/ps2400dpi150lpi/"))

In [ ]:
Image.open(download_blob(get_blobs( filter="data/148903/vps2400dpi150lpi/449.4c_300")[0]))

In [ ]:
Image.open(download_blob(get_blobs( filter="data/148903/ps2400dpi150lpi/449.4c_600")[0]))

In [ ]:
get_blobs( filter="data/148903/ps2400dpi150lpi/449.4c_600")

In [ ]:
processed_keys = []
relevant_keys = [key for key in results_by_img if len(results_by_img[key]['relevant_results']) > 0]

for key in tqdm(relevant_keys):
    relevant_tasks = [t for t in tasks if t['img_name'] == key and t['storage_type'] == 'azure']

    if len(relevant_tasks) > 0 and len(results_by_img[key]['relevant_results']) > 0:
        processed_keys.append(key)
        create_annotation( relevant_tasks[0], results_by_img[key]['relevant_results'] )

        if len(processed_keys) > 100:
            break

In [ ]:
processed_keys

# Datenbank updaten

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect(dotenv['DB_PATH'])

In [ ]:
results = get_results_of_project(2)

In [ ]:
local_results = [r for r in results if r['storage_type'] == 'local']
azure_results = [r for r in results if r['storage_type'] == 'azure']

In [ ]:
r = local_results[i]
matching_azure_results = [
    ra for ra in azure_results if (
    ra['img_name'] == r['img_name'] and
    ra['value']['x'] == r['value']['x'] and
    ra['value']['y'] == r['value']['y'] and
    ra['value']['width'] == r['value']['width'] and
    ra['value']['height'] == r['value']['height']
)]

In [ ]:
r['id'], matching_azure_results[0]

In [ ]:
for i in range(len(local_results)):
    r = local_results[i]

    if len([ra for ra in azure_results if ra['img_name'] == r['img_name']]):
        print(i)
        break

In [ ]:
r